In [1]:
import pandas as pd
from path import Path

In [8]:
def find_total_profits(df):
    # Convert open odds to integers
    df['Home_Open_Odds'] = df['Home_Open_Odds'].astype(int)
    df['Visitor_Open_Odds'] = df['Visitor_Open_Odds'].astype(int)

    # Pull in predicted win and predicted loss
    home_win = df[df['Predicted'] == 1]
    home_loss = df[df['Predicted'] == 0]

    # Grab instances where the predicted winner is the favorite to win
    home_win_fav = home_win[home_win['Home_Open_Odds'] < 0]
    home_loss_fav = home_loss[home_loss['Visitor_Open_Odds'] < 0]

    # Grab instances where the predicted winner is the underdog
    home_win_dog = home_win[home_win['Home_Open_Odds'] > 0]
    home_loss_dog = home_loss[home_loss['Visitor_Open_Odds'] > 0]

    # Grab instances where the favs were the actual winners and predicted winners
    home_win_fav_true = home_win_fav[home_win_fav['Actual'] == 1]
    home_win_fav_true['Home_Open_Odds'] = ((10000)/abs(home_win_fav_true['Home_Open_Odds']))
    home_loss_fav_true = home_loss_fav[home_loss_fav['Actual'] == 1]
    home_loss_fav_true['Visitor_Open_Odds'] = ((10000)/abs(home_loss_fav_true['Visitor_Open_Odds']))

    # Grab instances where the dogs were winners and predicted winners
    home_win_dog_true = home_win_dog[home_win_dog['Actual']==1]
    home_loss_dog_true = home_loss_dog[home_loss_dog['Actual']==0]

    # Grab instances where the dogs were losers, but predicted winners
    home_win_dog_false = home_win_dog[home_win_dog['Actual']==0]
    home_loss_dog_false = home_loss_dog[home_loss_dog['Actual']==1]

    # Grab instances where the favs were predicted winners but lost
    home_win_fav_false = home_win_fav[home_win_fav['Actual'] == 0]
    home_win_fav_false['Home_Open_Odds'] = -100
    home_loss_fav_false = home_loss_fav[home_loss_fav['Actual'] == 1]
    home_loss_fav_false['Visitor_Open_Odds'] = -100

    # Concatinate the dataframes
    bet_results_fav_win = pd.concat([home_win_fav_false, home_win_fav_true]) # Home Open Odds
    bet_results_fav_loss = pd.concat([home_loss_fav_false, home_loss_fav_true]) # Visitor Open Odds
    bet_results_dog_win = pd.concat([home_win_dog_true, home_win_dog_false]) # Home Open Odds
    bet_results_dog_loss = pd.concat([home_loss_dog_true, home_loss_dog_false]) # Visitor Open Odds

    # Sum up final wins/loss money lines
    fav_win_sum = bet_results_fav_win['Home_Open_Odds'].sum()
    fav_loss_sum = bet_results_fav_loss['Visitor_Open_Odds'].sum()
    dog_win_sum = bet_results_dog_win['Home_Open_Odds'].sum()
    dog_loss_sum = bet_results_dog_loss['Visitor_Open_Odds'].sum()
    final_sum = fav_win_sum + fav_loss_sum + dog_win_sum + dog_loss_sum
    return final_sum

In [10]:
year = input()
csv_name = '../Predictions_Vs_Actual/Predictions_Actual_' + year + '.csv'
csv_path = Path(csv_name)
df = pd.read_csv(csv_path, index_col = 'Date')
find_total_profits(df)

 2017


C:\Users\taylo\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\taylo\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\taylo\Anaconda\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

74581.63238477772